In [1]:
pip install -U spacy

  Using cached spacy-3.3.0-cp38-cp38-macosx_10_9_x86_64.whl (6.4 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.6
    Uninstalling spacy-3.1.6:
      Successfully uninstalled spacy-3.1.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-med7-trf 3.1.3.1 requires spacy<3.2.0,>=3.1.4, but you have spacy 3.3.0 which is incompatible.
en-core-med7-lg 3.1.3.1 requires spacy<3.2.0,>=3.1.4, but you have spacy 3.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl

  Using cached https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl (790.7 MB)
  Using cached spacy-3.1.6-cp38-cp38-macosx_10_9_x86_64.whl (6.1 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.0
    Uninstalling spacy-3.3.0:
      Successfully uninstalled spacy-3.3.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
import spacy
import os
from sklearn.metrics import f1_score
from spacy.scorer import Scorer
from spacy.training import Example
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm

In [9]:
training_list = []
def training_patterns(df):
    df.columns = ["entities", "data"]
    data = df.data.str.split('\t', expand=True)
    data.columns = ["labels", "value"]
    data = data[data.value.notnull()]
    patterns = np.asarray(data["value"])
    data = pd.DataFrame(data.labels.str.split(expand = True))
    if data.shape[1] != 3:
        data = data.iloc[:, [0,1,2]]
    data.columns = ["label", "start_index", "end_index"]
    data['patterns'] = patterns
    data = data[data.label != "Reason"]
    data = data[data.label != "ADE"]
    labels = np.asarray(data["label"])
    patterns = np.asarray(data["patterns"])
    
    for i in range(len(labels)):
        temp_dict = {}
        temp_dict["label"] = labels[i]
        temp_dict["pattern"] = patterns[i]
        training_list.append(temp_dict)
                                   
    return training_list
train_data = []
file_list=os.listdir(r"Training Data")
for file in file_list:
    if file.endswith(".ann"):
        df = pd.read_csv("Training Data/" + file, sep='^([^\s]*)\s', engine='python', header=None).drop(0, axis=1)
        train_data.append(training_patterns(df))
train_data = np.concatenate(train_data).ravel()


In [10]:
train_data

array([{'label': 'Drug', 'pattern': 'Plavix'},
       {'label': 'Strength', 'pattern': '0.5 mg'},
       {'label': 'Form', 'pattern': 'Tablet'}, ...,
       {'label': 'Duration', 'pattern': 'two week'},
       {'label': 'Route', 'pattern': 'IV'},
       {'label': 'Route', 'pattern': 'IV'}], dtype=object)

In [11]:
med7 = spacy.load("en_core_med7_lg")

ruler = med7.add_pipe("entity_ruler")
patterns = [{"label": "DRUG", "pattern": "prednsone"},
                {"label": "DRUG", "pattern": "aspirin"},
                {"label": "DRUG", "pattern": "vitamin b"},
                {"label": "DRUG", "pattern": "flagyl"},
                {"label": "DRUG", "pattern": "lisinopril"},
                {"label": "FORM", "pattern": "solution"},
                {"label": "FORM", "pattern": "tablet"},
                {"label": "FORM", "pattern": "capsule"},
                {"label": "FORM", "pattern": "puff"},
                {"label": "FORM", "pattern": "adhesive patch"},
                {"label": "FORM", "pattern": "disk with device"},
                {"label": "STRENGTH", "pattern": "50mg/2ml"},
                {"label": "STRENGTH", "pattern": "5mg"},
                {"label": "STRENGTH", "pattern": "100 unit/ml"},
                {"label": "STRENGTH", "pattern": "0.05%"},
                {"label": "STRENGTH", "pattern": "25-50mg"},
                {"label": "DURATION", "pattern": "for 3 days"},
                {"label": "DURATION", "pattern": "7 days"},
                {"label": "DURATION", "pattern": "chronic"},
                {"label": "DURATION", "pattern": "x5 days"},
                {"label": "DURATION", "pattern": "for 5 or more days"},
                {"label": "ROUTE", "pattern": "PO"},
                {"label": "ROUTE", "pattern": "iv"},
                {"label": "ROUTE", "pattern": "gtt"},
                {"label": "ROUTE", "pattern": "nasal canula"},
                {"label": "ROUTE", "pattern": "injection"},
                {"label": "DOSAGE", "pattern": "1-2"},
                {"label": "DOSAGE", "pattern": "sliding scale"},
                {"label": "DOSAGE", "pattern": "taper"},
                {"label": "DOSAGE", "pattern": "bolus"},
                {"label": "DOSAGE", "pattern": "thirty (30) ml"},
                {"label": "FREQUENCY", "pattern": "once a day"},
                {"label": "FREQUENCY", "pattern": "b.i.d."},
                {"label": "FREQUENCY", "pattern": "prn"},
                {"label": "FREQUENCY", "pattern": "q6h"},
                {"label": "FREQUENCY", "pattern": "hs"},
                {"label": "FREQUENCY", "pattern": "every six (6) hours as needed"}]
ruler.add_patterns(train_data)

In [12]:
patterns_list = []
def testing_patterns(df):
    df.columns = ["entities", "data"]
    data = df.data.str.split('\t', expand=True)
    data.columns = ["labels", "value"]
    data = data[data.value.notnull()]
    patterns = np.asarray(data["value"])
    data = pd.DataFrame(data.labels.str.split(expand = True))
    if data.shape[1] != 3:
        data = data.iloc[:, [0,1,2]]
    data.columns = ["label", "start_index", "end_index"]
    data['patterns'] = patterns
    data = data[data.label != "Reason"]
    data = data[data.label != "ADE"]
    labels = np.asarray(data["label"])
    patterns = np.asarray(data["patterns"])
    
    for i in range(len(labels)):
        temp_dict = {}
        temp_dict['label'] = labels[i]
        temp_dict['pattern'] = patterns[i]
        patterns_list.append(temp_dict)
                                   
    return patterns_list
val_data = []
for file in os.listdir():
    if file.endswith(".ann"):
        df = pd.read_csv(file, sep='^([^\s]*)\s', engine='python', header=None).drop(0, axis=1)
        val_data.append(testing_patterns(df))
val_data = np.concatenate(val_data).ravel()

val_labels = []
val_patterns = []
unique_val_data = []

for i in range(len(val_data)):
    if val_data[i] not in unique_val_data:
        unique_val_data.append(val_data[i])

for i in range(len(unique_val_data)):
    val_labels.append(unique_val_data[i]['label'].lower())
    val_patterns.append(unique_val_data[i]['pattern'].lower())

In [13]:
%%time

med7 = spacy.load("en_core_med7_lg")
d_test = {}
y_hat = []

keyList = ["DOSAGE", "DRUG", "DURATION", "FORM", "FREQUENCY", "ROUTE", "STRENGTH"]

for i in keyList:
    d_test[i] = []
for file in os.listdir():
    if file.endswith(".txt"):
        file1 = open(file, 'r')

        for line in file1:
            doc = med7(str(line))
            [d_test[ent.label_].append(ent.text) for ent in doc.ents]
            
unique_d_test = {}
for i in keyList:
    unique_d_test[i] = []

for key in d_test:
    for i in d_test[key]:
        if i not in unique_d_test[key]:
            unique_d_test[key].append(i)
test_patterns = []  
test_labels = []
for key in unique_d_test:
    for i in unique_d_test[key]:
        test_patterns.append(i.lower())
        test_labels.append(key.lower())

CPU times: user 7min 53s, sys: 14.1 s, total: 8min 7s
Wall time: 8min 15s


In [25]:
d_test

{'DOSAGE': ['high dose',
  'bolus',
  'one',
  'One (1)',
  'Five (5) ML',
  'Six (6)',
  'One (1)',
  '0.5',
  'One (1)',
  'Two (2)',
  '1-2',
  'One (1)',
  'One (1)',
  'Two (2)',
  'One (1)',
  '3',
  'Three (3)',
  '0.5-1',
  'tapered',
  'one unit',
  '1',
  'One (1)',
  'Two\n',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  '1.5',
  '1500cc',
  'bolus',
  '7d',
  'sliding scale',
  '1',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'One (1)',
  'Thirty (30) ML',
  'One (1)',
  'One (1)',
  'One (1)',
  '0.5',
  'One (1) ml',
  '2-10 units',
  'sliding scale',
  '2u',
  '6u',
  'long',
  'One (1)',
  'One (1)',
  'One (1)',
  'Two (2)',
  '0.5',
  'One (1)',
  'One (1)',
  'One (1)',
  '0.5',
  'Five (5) ML',
  'One (1) ML',
  'One (1)',
  'Two (2)',
  'Two (2)',
  'One (1) ML',
  'High dose',
  'sliding scale',
  'One (1)',
  'One (1)',
  'One (1)

In [14]:
potential_missing_count = 0
potential_spurious_count = 0
correct_count = 0
potential_missing_patterns = []
potential_spurious_patterns = []

for i in range(len(test_patterns)):
    pat_to_test = test_patterns[i].lower()
    if pat_to_test not in val_patterns:
        potential_spurious_count+=1
        potential_spurious_patterns.append(pat_to_test)

In [15]:
potential_incomplete = 0
potential_extra = 0
extra_patterns = []
incomplete_patterns = []
for i in range(len(potential_spurious_patterns)):
    potential_missing_pattern = potential_spurious_patterns[i]
    for j in range(len(val_patterns)):
        if potential_missing_pattern in val_patterns[j]:
            incomplete_patterns.append(potential_missing_pattern)
        if val_patterns[j] in potential_missing_pattern:
            extra_patterns.append(potential_missing_pattern)
            
unique_incomplete_patterns = []
unique_extra_patterns = []

incomplete = 0
extra = 0

for i in range(len(incomplete_patterns)):
    if incomplete_patterns[i] not in unique_incomplete_patterns:
        unique_incomplete_patterns.append(incomplete_patterns[i])
        incomplete += 1
        
for i in range(len(extra_patterns)):
    if extra_patterns[i] not in unique_extra_patterns:
        unique_extra_patterns.append(extra_patterns[i])
        extra += 1
        
partial = incomplete + extra
spurious_missing = potential_spurious_count - partial
spurious = int(spurious_missing/2)
missing = spurious

In [16]:
correct = 0
correct_patterns = []
correct_labels = []

for i in range(len(test_patterns)):
    pattern_to_test = test_patterns[i]
    label_to_test = test_labels[i]
    if pattern_to_test in val_patterns:
        index = val_patterns.index(pattern_to_test)
        if label_to_test.lower() == val_labels[index].lower():
            correct += 1
            correct_patterns.append(pattern_to_test)
            correct_labels.append(label_to_test)
            
incorrect = len(test_patterns) - (correct + missing + spurious + partial)

In [17]:
possible = correct + incorrect + partial + missing
actual = correct + incorrect + partial + spurious
precision_lenient = (correct + partial)/actual
precision_strict = correct/actual
recall_lenient = (correct + partial)/possible
recall_strict = correct/possible

len_f1 = 2 * (precision_lenient * recall_lenient)/(precision_lenient + recall_lenient)
strict_f1 = 2 * (precision_strict * recall_strict)/(precision_strict + recall_strict)

In [18]:
len_f1

0.9877492877492877

In [19]:
strict_f1

0.7558404558404559

In [20]:
Counter(correct_labels)

Counter({'dosage': 150,
         'drug': 1493,
         'duration': 88,
         'form': 142,
         'frequency': 267,
         'route': 89,
         'strength': 424})

In [21]:
Counter(test_labels)

Counter({'dosage': 253,
         'drug': 1788,
         'duration': 131,
         'form': 158,
         'frequency': 497,
         'route': 114,
         'strength': 530})

In [22]:
correct/len(test_patterns)

0.7643330452319216